#  Modelo de ZeeHB

Sample of notebook for specific model

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import os, sys, inspect
import commands
import hep as hp

In [3]:
def func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam3, lam6, lam7, lam8, lam9, lam10, lamh):
    M12 = M1*M1; M22 = M2*M2; MAo2 = MAo*MAo; Mho2 = Mho*Mho; MHo2 = MHo*MHo;
    k = s2phi*np.log(M22/M12)/(4.*np.pi)**2
    phi = 0.5*np.arcsin(s2phi)
    Mu  = (M22-M12)*s2phi/(np.sqrt(2.0)*v)
    MH2 = M12*(np.sin(phi))**2+M22*(np.cos(phi))**2
    M332 = M12*(np.cos(phi))**2+M22*(np.sin(phi))**2
    lam1 = 0.5*(MHo2+Mho2-np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2))/(v**2)
    Mu22 = MH2-0.5*lam3*(v**2)
    Muh2 = M332-0.5*lam8*(v**2)
    lam4 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2)+MHo2+Mho2+2.0*(MAo2-2.0*MH2))/(v**2)
    lam5 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2)+MHo2+Mho2-2.0*MAo2)/(v**2)    
    return k, Mu, Muh2, Mu22, lam1, lam4, lam5

In [10]:
import hep as hp
def Kappa(s2phi,M1,M2):
    return s2phi*np.log(M2**2/M1**2)/(4.*np.pi)**2

def Inverse_Zee_Matrix(kappa,t12,t13,t23,ml,IH=False,sgnm2=-1,sgnml=1):
    """Inverse neutrino mass matrix normalized by kappa from
        Mnu_diag=U^T.(kappa Y).U
        ,
        (Inverse_Zee_Matrix) = U. Mnu_diag.U^T/kappa
        sgnm2=-1 guarantees real Yukawa couplings
        
    requires hep.py:
       https://github.com/restrepo/BSM-Toolbox/blob/master/tests/hep.py
    """
    if not IH:
        mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(CL=3,IH=False,mnu1in=1E-5*1E-9)
        theta12[1] = t12
        theta13[1] = t13
        theta23[1] = t23
        U=hp.UPMNS(theta12[1],theta13[1],theta23[1])
        m1=ml; m2=sgnm2*np.sqrt(Dm21_2[1]+m1**2); m3=sgnml*np.sqrt(Dm3l_2[1]+m1**2)
    else:
        mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(CL=3,IH=True,mnu1in=1E-5*1E-9)
        #DEBUG signs
        m3=ml; m2=sgnm2*np.sqrt(-Dm3l_2[1]+m3**2); m1=sgnml*np.sqrt(m2**2-Dm21_2[1])
        theta12[1] = t12
        theta13[1] = t13
        theta23[1] = t23
        U=hp.UPMNS(theta12[1],theta13[1],theta23[1])
        
    return np.dot( np.dot( U,np.diag([m1,m2,m3]) ), U.transpose() )/kappa

def get_yukawas(X,q00=1e-03,q01=1e-3,q11=1e-3,q20=1e-3,q21=-1e-3,q22=0,m_e=0.5109989461e-3,m_mu=0.1056583745,m_tau=1.77686):
    """
    Get the O and f full Yukawa mass matrices which define the neutrino mass matrix in the Zee model
      Mnu=kappa*(O.M_lep.f^T+f.M_lep.O^T)
    where M_lep is the diagonal matrix with the charged leptons
    
    The input X is the output of the `def Inverse_Zee_Matrix(...)` in this module
    """
    import numpy.lib.scimath as sc # .sqrt -> returns complex for sqrt(negative real)
    ml=np.array([m_e,m_mu,m_tau])
    Q=np.zeros((3,3))
    f=np.zeros((3,3))
    
    Q[0,0]=q00; Q[0,1]=q01; Q[1,1]=q11; Q[2,0]=q20; Q[2,1]=q21; Q[2,2]=q22
    
    
    f[0,1] = (ml[0]*(Q[0,1]*Q[2,0]-Q[0,0]*Q[2,1])*X[0,0]+ml[2]*(Q[2,1]*(Q[2,2]*X[0,0]-2*Q[0,2]*X[0,2])+Q[0,1]*Q[0,2]*X[2,2]))/\
    (2.*ml[1]*(ml[0]*Q[0,1]*(Q[0,1]*Q[2,0]-Q[0,0]*Q[2,1])+ml[2]*Q[2,1]*(-(Q[0,2]*Q[2,1])+Q[0,1]*Q[2,2])))

    f[0,2] = (Q[2,1]**2*X[0,0]-2*Q[0,1]*Q[2,1]*X[0,2]+Q[0,1]**2*X[2,2])/\
    (2.*(ml[0]*Q[0,1]*(-(Q[0,1]*Q[2,0]) + Q[0,0]*Q[2,1])+ml[2]*Q[2,1]*(Q[0,2]*Q[2,1] - Q[0,1]*Q[2,2])))

    f[1,2] = (ml[2]*(Q[0,2]*Q[2,1]-Q[0,1]*Q[2,2])*X[2,2]+ml[0]*(Q[2,0]*(Q[2,1]*X[0,0]-2*Q[0,1]*X[0,2])+Q[0,0]*Q[0,1]*X[2,2]))/\
    (2.*ml[1]*(ml[0]*Q[0,1]*(Q[0,1]*Q[2,0]-Q[0,0]*Q[2,1])+ml[2]*Q[2,1]*(-(Q[0,2]*Q[2,1])+Q[0,1]*Q[2,2])))

    Q[1,0] = -((ml[0]**3*Q[0,0]*(-(Q[0,1]*Q[2,0])+Q[0,0]*Q[2,1])*X[0,0]*(Q[2,0]*(Q[2,1]*X[0,0]-2*Q[0,1]*X[0,2])+\
    Q[0,0]*Q[0,1]*X[2,2])+ml[2]*(Q[0,2]*Q[2,1]-Q[0,1]*Q[2,2])*(ml[2]**2*Q[0,2]*(Q[0,2]*Q[2,1]-\
    Q[0,1]*Q[2,2])*X[2,2]**2+ml[1]**2*Q[2,1]*X[1,1]*(Q[2,1]**2*X[0,0] - 2*Q[0,1]*Q[2,1]*X[0,2]+\
    Q[0,1]**2*X[2,2])+ml[1]*ml[2]*X[2,2]*(2*Q[2,1]*(Q[0,2]*Q[2,1]-Q[0,1]*Q[2,2])*X[0,1]+Q[1,1]*(Q[2,1]*(Q[2,2]*X[0,0]-\
    2*Q[0,2]*X[0,2])+Q[0,1]*Q[0,2]*X[2,2])))+ml[0]**2*(ml[1]*(Q[0,1]*Q[2,0]-Q[0,0]*Q[2,1])*(-(Q[1,1]*X[0,0])+\
    2*Q[0,1]*X[0,1])*(Q[2,0]*(-(Q[2,1]*X[0,0])+2*Q[0,1]*X[0,2])-Q[0,0]*Q[0,1]*X[2,2])+\
    ml[2]*(Q[0,0]*Q[2,2]*X[0,0]*(-2*Q[0,0]*Q[0,1]*Q[2,1]*X[2,2]+Q[2,0]*(-(Q[2,1]**2*X[0,0])+2*Q[0,1]*Q[2,1]*X[0,2]+\
    Q[0,1]**2*X[2,2]))+Q[0,2]*(Q[2,0]**2*(Q[2,1]*X[0,0]-2*Q[0,1]*X[0,2])**2+Q[0,0]**2*Q[2,1]*(Q[2,1]*X[0,0]+\
    2*Q[0,1]*X[0,2])*X[2,2]-2*Q[0,0]*Q[2,0]*X[0,2]*(-(Q[2,1]**2*X[0,0]) + 2*Q[0,1]*Q[2,1]*X[0,2]+\
    Q[0,1]**2*X[2,2]))))-ml[0]*(ml[1]**2*Q[0,1]*(Q[0,1]*Q[2,0]-Q[0,0]*Q[2,1])*X[1,1]*(Q[2,1]**2*X[0,0]-\
    2*Q[0,1]*Q[2,1]*X[0,2]+Q[0,1]**2*X[2,2])-ml[2]**2*(Q[0,2]*Q[2,1]-Q[0,1]*Q[2,2])*X[2,2]*\
    (-(Q[0,0]*Q[2,1]*Q[2,2]*X[0,0])+Q[0,2]*(2*Q[2,0]*(Q[2,1]*X[0,0]-2*Q[0,1]*X[0,2])+\
    Q[0,0]*(2*Q[2,1]*X[0,2]+Q[0,1]*X[2,2])))+ml[1]*ml[2]*(-2*(Q[0,2]*Q[2,1]-Q[0,1]*Q[2,2])*X[0,1]*\
    (2*Q[0,0]*Q[0,1]*Q[2,1]*X[2,2]+Q[2,0]*(Q[2,1]**2*X[0,0]-2*Q[0,1]*Q[2,1]*X[0,2]-Q[0,1]**2*X[2,2]))+\
    Q[1,1]*(Q[0,0]*X[2,2]*(-2*Q[0,1]*Q[2,1]*Q[2,2]*X[0,0]+Q[0,2]*(Q[2,1]**2*X[0,0]+2*Q[0,1]*Q[2,1]*X[0,2]-\
    Q[0,1]**2*X[2,2]))+Q[2,0]*(Q[2,1]**2*X[0,0]*(-(Q[2,2]*X[0,0])+2*Q[0,2]*X[0,2])+Q[0,1]**2*(Q[2,2]*X[0,0]+\
    2*Q[0,2]*X[0,2])*X[2,2]+2*Q[0,1]*Q[2,1]*(Q[2,2]*X[0,0]*X[0,2]-Q[0,2]*(2*X[0,2]**2+X[0,0]*X[2,2])))))))/\
    (ml[0]*ml[1]*(Q[2,1]**2*X[0,0]-2*Q[0,1]*Q[2,1]*X[0,2]+Q[0,1]**2*X[2,2])*(ml[0]*(-(Q[0,1]*Q[2,0])+\
    Q[0,0]*Q[2,1])*X[0,0]-ml[2]*(Q[2,1]*(Q[2,2]*X[0,0]-2*Q[0,2]*X[0,2])+Q[0,1]*Q[0,2]*X[2,2]))))

    Q[1,2] = (ml[0]**2*Q[0,0]*(-(Q[0,1]*Q[2,0])+Q[0,0]*Q[2,1])*X[0,0]+ml[0]*(-(ml[1]*(Q[0,1]*Q[2,0]-\
    Q[0,0]*Q[2,1])*(-(Q[1,1]*X[0,0])+2*Q[0,1]*X[0,1]))+ml[2]*(-(Q[0,0]*Q[2,1]*Q[2,2]*X[0,0])+\
    Q[0,2]*(2*Q[0,0]*Q[2,1]*X[0,2]+Q[2,0]*(Q[2,1]*X[0,0]-2*Q[0,1]*X[0,2]))))+\
    ml[2]*(ml[2]*Q[0,2]*(Q[0,2]*Q[2,1]-Q[0,1]*Q[2,2])*X[2,2]+ml[1]*(2*Q[2,1]*(Q[0,2]*Q[2,1]-\
    Q[0,1]*Q[2,2])*X[0,1]+Q[1,1]*(Q[2,1]*(Q[2,2]*X[0,0]-2*Q[0,2]*X[0,2])+Q[0,1]*Q[0,2]*X[2,2]))))/\
    (ml[1]*ml[2]*(Q[2,1]**2*X[0,0] - 2*Q[0,1]*Q[2,1]*X[0,2] + Q[0,1]**2*X[2,2]))

    Q[0,2] = (ml[2]**2*Q[2,2]*X[0,2]*X[2,2]+ml[2]*(2*X[0,2]*(ml[0]*Q[2,0]*X[0,2]+ml[1]*Q[2,1]*X[1,2])-\
    (ml[0]*(Q[2,0]*X[0,0]+Q[0,0]*X[0,2])+ml[1]*(Q[2,1]*X[0,1] + Q[0,1]*X[1,2]))*X[2,2])+\
    np.sqrt(ml[2]**2*((-2*X[0,2]*(ml[0]*Q[2,0]*X[0,2]+ml[1]*Q[2,1]*X[1,2])+(ml[0]*Q[2,0]*X[0,0]+\
    (ml[0]*Q[0,0] - ml[2]*Q[2,2])*X[0,2]+ml[1]*(Q[2,1]*X[0,1]+Q[0,1]*X[1,2]))*X[2,2])**2-\
    X[2,2]**2*(ml[2]**2*Q[2,2]**2*X[0,0]*X[2,2]+ml[0]**2*X[0,0]*(Q[2,0]**2*X[0,0]-2*Q[0,0]*Q[2,0]*X[0,2]+\
    Q[0,0]**2*X[2,2])+ml[1]**2*X[1,1]*(Q[2,1]**2*X[0,0]-2*Q[0,1]*Q[2,1]*X[0,2]+Q[0,1]**2*X[2,2])+\
    2*ml[1]*ml[2]*Q[2,2]*(Q[2,1]*X[0,0]*X[1,2]-Q[0,1]*X[0,1]*X[2,2])+2*ml[0]*(ml[2]*Q[2,2]*X[0,0]*(Q[2,0]*X[0,2]-\
    Q[0,0]*X[2,2])+ml[1]*(Q[2,0]*(X[0,1]*(Q[2,1]*X[0,0]-2*Q[0,1]*X[0,2])+Q[0,1]*X[0,0]*X[1,2])+\
    Q[0,0]*(-(Q[2,1]*X[0,0]*X[1,2])+Q[0,1]*X[0,1]*X[2,2])))))))/(ml[2]**2*X[2,2]**2)    
    
    f[1,0]=-f[0,1]; f[2,0]=-f[0,2]; f[2,1]=-f[1,2]
    
    return Q,f

def test_Zee(q00,q01,q11,q20,q21,q22):
    m_e=0.5109989461e-3;m_mu=0.1056583745;m_tau=1.77686
    mlep=np.diag([m_e,m_mu,m_tau])
    m1=0 # lightest neutrino
    s2phi=0.014; M1=200.; M2=300.
    t12=33.56*180.0/np.pi
    t13=8.49*180.0/np.pi
    t23=41.6*180.0/np.pi
    X=Inverse_Zee_Matrix(Kappa(s2phi,M1,M2),t12,t13,t23,m1)
    O,f=get_yukawas(X,q00,q01,q11,q20,q21,q22) #Use default input values
    Mnu=Kappa(s2phi,M1,M2)*( np.dot( np.dot(O,mlep),f.transpose() )+np.dot( np.dot(f,mlep),O.transpose()) )
    Mnu_diag,U=np.linalg.eig(Mnu)
    lo=np.argsort(np.abs(Mnu_diag))
    Mnu_diag=np.array([Mnu_diag[lo[0]],Mnu_diag[lo[1]],Mnu_diag[lo[2]]])
    U=np.matrix(U)
    U=np.asarray(np.hstack((U[:,lo[0]],U[:,lo[1]],U[:,lo[2]])))
    
    mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(CL=3,IH=False,mnu1in=1E-5*1E-9)
    m2=np.sqrt(Dm21_2[1]+m1**2); m3=np.sqrt(Dm3l_2[1]+m1**2)
    theta12[1] = t12
    theta13[1] = t13
    theta23[1] = t23
    
    np.testing.assert_array_almost_equal(np.abs(U),\
        np.abs( hp.UPMNS(theta12[1],theta13[1],theta23[1]) ) )
    
    return np.testing.assert_almost_equal( np.abs(Mnu_diag),np.abs([m1,m2,m3]) ) 

In [13]:
test_Zee(q00=-4.255172e-03,q01=2.412983e-05,q11=3.109212e-11,q20=-4.987462e-04,q21=-2.227795e-05,q22=3.246787e-08)

AssertionError: 
Arrays are not almost equal to 6 decimals

(mismatch 100.0%)
 x: array([[ 0.001128,  0.026256,  0.999655],
       [ 0.651431,  0.758427,  0.020655],
       [ 0.758707,  0.651229,  0.016248]])
 y: array([[ 0.858821,  0.166644,  0.484414],
       [ 0.282197,  0.6353  ,  0.718859],
       [ 0.427542,  0.754072,  0.498582]])

In [14]:
hp.neutrino_data(IH=True)

(1.0000000000000002e-14,
 array([  7.03000000e-23,   7.50000000e-23,   8.09000000e-23]),
 array([ -2.39900000e-21,  -2.51400000e-21,  -2.63500000e-21]),
 array([ 0.54768432,  0.5857325 ,  0.628144  ]),
 array([ 0.67718775,  0.87266463,  0.92676983]),
 array([ 0.14014994,  0.14817845,  0.1558579 ]),
 array([ 2.53072742,  4.83456203,  6.82423738]))

In [15]:
import numpy.lib.scimath as sc

In [16]:
s2phi=0.014; M1=200.; M2=300.
kappa=Kappa(s2phi,M1,M2)

In [20]:
m3=1E-15
t12=33.56*180.0/np.pi
t13=8.49*180.0/np.pi
t23=41.6*180.0/np.pi
X=Inverse_Zee_Matrix(Kappa(s2phi,M1,M2),t12,t13,t23,m3)
X

array([[  1.60643366e-07,  -2.56089533e-07,   1.53632126e-07],
       [ -2.56089533e-07,   3.12494370e-07,  -3.08166110e-07],
       [  1.53632126e-07,  -3.08166110e-07,   1.05217268e-07]])

In [23]:
O,f=get_yukawas(X,q00=-4.255172e-03,q01=2.412983e-05,q11=3.109212e-11,q20=-4.987462e-04,q21=-2.227795e-05,q22=3.246787e-08)

In [24]:
m_e=0.5109989461e-3;m_mu=0.1056583745;m_tau=1.77686
mlep=np.diag([m_e,m_mu,m_tau])
Mnu=Kappa(s2phi,M1,M2)*( np.dot( np.dot(O,mlep),f.transpose() )+np.dot( np.dot(f,mlep),O.transpose()) )
Mnu_diag,U=np.linalg.eig(Mnu)

In [25]:
lo=np.argsort(np.abs(Mnu_diag))
Mnu_diag=np.array([Mnu_diag[lo[1]],Mnu_diag[lo[2]],Mnu_diag[lo[0]]])
U=np.matrix(U)
U=np.asarray(np.hstack((U[:,lo[1]],U[:,lo[2]],U[:,lo[0]])))

In [26]:
Mnu_diag # m

array([  6.36630929e-11,   7.48812271e-10,  -3.41055336e-11])

In [27]:
U

array([[ 0.02625653,  0.9996546 ,  0.00112815],
       [-0.75842889,  0.02065576, -0.6514284 ],
       [-0.6512267 ,  0.01624863,  0.75870928]])

In [28]:
mltmp,Dm21_2,Dm3l_2,theta12,theta23,theta13,delta=hp.neutrino_data(IH=True)
#m2=np.sqrt(Dm21_2[1]+m1**2); m3=np.sqrt(Dm3l_2[1]+m1**2)
np.abs( hp.UPMNS(theta12[1],theta13[1],theta23[1]) )

array([[ 0.8241757 ,  0.54675204,  0.14763679],
       [ 0.44958339,  0.47311888,  0.75764986],
       [ 0.34439685,  0.69081165,  0.63574372]])

[![Home](http://www.incredimail.com/images/nav%20bar/home-icon.png)](./) 
[Jupyter home](./draft.pdf)